In [1]:
#check goldfinger and goldeneye.
#why is bond not protagonist?
#kaggle data is not reliable in terms of character importance, need another method to select protagonists

from llm_annotate import chunk_text, annotate, custom_openai
import pandas as pd
import os

In [2]:
#load kaggle data
kaggle_data = pd.read_csv("data/kaggle_data.csv")

In [3]:
repeat_chunking = False
if repeat_chunking:
    for fn in kaggle_data.script_filename:
        with open(f"data/screenplay_data/data/raw_texts/raw_texts/{fn}", "r", encoding="utf-8") as f:
            try:
                script = f.read()
            except Exception as e:
                try:
                    with open(f"data/screenplay_data/data/raw_texts/raw_texts/{fn}", "r", encoding="iso-8859-1") as f:
                        script = f.read()
                except Exception as e2:
                    print(f"again Error reading {fn}: {e2}")
                    raise e2
        chunk_text(script, outputfile=f"data/chunks/{fn.replace('.txt', '.json')}", verbose=False)

In [4]:
traits= {
    "warmth": {
        "trait_explanation": "A character’s warmth refers to their tendency to be good-natured, trustworthy, tolerant, friendly, and sincere. ",
        "positive_examples": "Organizes a get-together, admits to insecurity.",
        "negative_examples": "Rejects someone, boasts."
    },
    "competence": {
        "trait_explanation": "A character’s competence refers to their tendency to behave capably, skillfully, intelligently, and confidently.",
        "positive_examples": "Wins something, takes a calculated risk.",
        "negative_examples": "Embarrasses themselves, fails to understand something."
    }
}

In [ ]:
# for i in range(10): #kaggle_data.shape[0]
#     #if annotation file exists, skip
#     current_file = kaggle_data.script_filename.iloc[i]
#     if os.path.exists(f"data/annotations/{current_file.replace('.txt', '.json')}"):
#         continue
#     print(f"{i+1}/{kaggle_data.shape[0]}")
#     title = kaggle_data.title.iloc[i]
#     protagonist = kaggle_data.character_name.iloc[i]
#     annotate(chunkfile=f"data/chunks/{current_file.replace('.txt', '.json')}", outputfile=f"data/annotations/{current_file.replace('.txt', '.json')}", book_title=title, traits=traits, target_characters=[protagonist], model="gpt-4.1-mini")

In [5]:
def annotate_one(i):
    current_file = kaggle_data.script_filename.iloc[i]
    out_path = f"data/annotations/{current_file.replace('.txt', '.json')}"
    if os.path.exists(out_path):
        return f'Skipping {i} '
    print(f"{i+1}/{kaggle_data.shape[0]}")
    title = kaggle_data.title.iloc[i]
    protagonist = kaggle_data.character_name_search.iloc[i]
    annotate(
        chunkfile=f"data/chunks/{current_file.replace('.txt', '.json')}",
        outputfile=out_path,
        book_title=title,
        traits=traits,
        target_characters=[protagonist],
        model=custom_openai
    )
    return f"Done {i}: {current_file}"

In [6]:
from concurrent.futures import ThreadPoolExecutor, as_completed
with ThreadPoolExecutor(max_workers=6) as executor:
    futures = [executor.submit(annotate_one, i) for i in range(kaggle_data.shape[0])]  # or range(kaggle_data.shape[0])
    for future in as_completed(futures):
        print(future.result())

Skipping 5 
Skipping 2 
Skipping 12 
Skipping 4 
Skipping 1 
Skipping 9 
Skipping 10 
Skipping 0 
Skipping 8 
Skipping 7 
Skipping 11 
Skipping 6 
Skipping 3 
Skipping 13 
Skipping 19 
Skipping 20 
Skipping 21 
Skipping 22 
Skipping 23 
Skipping 24 
Skipping 25 
Skipping 26 
Skipping 27 
Skipping 28 
Skipping 29 
Skipping 30 
Skipping 31 
Skipping 32 
Skipping 33 
Skipping 34 
Skipping 35 
Skipping 36 
Skipping 37 
Skipping 38 
Skipping 39 
Skipping 40 
Skipping 41 
Skipping 42 
Skipping 43 
Skipping 44 
Skipping 45 
Skipping 46 
Skipping 47 
Skipping 48 
Skipping 49 
Skipping 50 
Skipping 51 
Skipping 52 
Skipping 53 
Skipping 54 
Skipping 55 
Skipping 56 
Skipping 57 
Skipping 58 
Skipping 59 
Skipping 60 
Skipping 61 
Skipping 62 
Skipping 63 
Skipping 64 
Skipping 65 
Skipping 66 
Skipping 67 
Skipping 68 
Skipping 69 
Skipping 70 
Skipping 71 
Skipping 72 
Skipping 73 
Skipping 74 
Skipping 75 
Skipping 76 
Skipping 77 
Skipping 78 
Skipping 79 
Skipping 80 
Skipping 81 
Skipping 

In [ ]:
from llm_annotate import score_annotations

score_annotations(annotation_file="data/annotations/10 Things I Hate About You_0147800.json", chunkfile = "data/chunks/10 Things I Hate About You_0147800.json", outputfile="trash.jsonl")

In [ ]:
#PLOT DISTRIBUTION OF binary columns Drama,Thriller,Comedy,Action,Crime,Romance,Adventure,Sci-Fi,Mystery,Horror,Fantasy,Biography
import matplotlib.pyplot as plt
genre_cols = ["Drama","Thriller","Comedy","Action","Crime","Romance","Adventure","Sci-Fi","Mystery","Horror","Fantasy","Biography"]
genre_counts = kaggle_data[genre_cols].sum().sort_values(ascending=False)
plt.figure(figsize=(10,6))
genre_counts.plot(kind='bar')
plt.title("Distribution of Movie Genres in Kaggle Dataset")
plt.xlabel("Genre")
plt.ylabel("Number of Movies")
plt.tight_layout()
